[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/Retrieval-based-Voice-Conversion-WebUI-colab/blob/main/retrieval_based_voice_conversion_webui_colab.ipynb)

In [ ]:
%cd /content
%env TF_CPP_MIN_LOG_LEVEL=1
!apt -y install -qq aria2
!pip install -q faiss-gpu fairseq gradio ffmpeg ffmpeg-python praat-parselmouth pyworld numpy==1.23.5 numba==0.56.4 librosa==0.9.2 mega.py gdown tensorboardX
!git clone https://huggingface.co/audo/Retrieval-based-Voice-Conversion-WebUI
%cd /content/Retrieval-based-Voice-Conversion-WebUI
!mkdir -p pretrained uvr5_weights
!python easy-infer.py --colab